# Env

In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
url = "http://127.0.0.1:8800"

# Mobile Base Control

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
pos = get_mobile_joint_position()
RESULT = {
    "mobile_joint_position": pos
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
result = set_mobile_target_joint([-1.0, -1.0, 0])
RESULT = result
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
grid_map = get_grid_map()
path = plan_mobile_path([1.0, -0.5])
if path is not None:
    for p in path:
        set_mobile_target_joint(p)
RESULT = {
    "grid_map": grid_map,
    "path": path
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
result = response.json()["result"]
grid_map = np.array(result["grid_map"])
path = result["path"]

print("Path length:", len(path))

grid_size = 0.1
floor_pos = [0.0, 0.0]

# Grid dimensions from actual grid_map
height, width = grid_map.shape  # (53, 60)

# Calculate actual floor bounds
floor_half_x = (width * grid_size) / 2  # 60 * 0.1 / 2 = 3.0
floor_half_y = (height * grid_size) / 2  # 53 * 0.1 / 2 = 2.65

floor_min_x = floor_pos[0] - floor_half_x
floor_max_y = floor_pos[1] + floor_half_y

# Mark path on grid_map with value 2
for waypoint in path:
    path_x, path_y = waypoint[0], waypoint[1]
    path_grid_j = int(np.floor((path_x - floor_min_x) / grid_size))
    path_grid_i = int(np.floor((floor_max_y - path_y) / grid_size))
    
    # Clamp to grid bounds
    path_grid_i = max(0, min(height - 1, path_grid_i))
    path_grid_j = max(0, min(width - 1, path_grid_j))
    
    grid_map[path_grid_i, path_grid_j] = 2

plt.imshow(grid_map, cmap='viridis')
plt.axis('off')
plt.show()

# Arm Joint Control

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
pos = get_arm_joint_position()
RESULT = {
    "arm_joint_position": pos
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
arm_joint_position = response.json()["result"]["arm_joint_position"]
arm_joint_position

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
result = set_arm_target_joint([-0.011, -1.032, 0.051, -2.260, 0.063, 1.523, 0.675])
RESULT = result
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

# End Effector Control

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
pos, ori = get_ee_position()
RESULT = {
    "ee_position": {
        "pos": pos,
        "ori": ori
    }
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
ee_position = response.json()["result"]["ee_position"]
ee_position

In [ ]:
ee_target_position = np.array(ee_position["pos"]) + np.array([0.1, 0.0, 0.0 ])
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": f"""
result = set_ee_target_position(({ee_target_position.tolist()}))
RESULT = result
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response

# Grid Map

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
grid_map = get_grid_map()
mobile_pos = get_mobile_joint_position()
RESULT = {
    "grid_map": grid_map,
    "mobile_pos": mobile_pos
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
result = response.json()["result"]
grid_map = np.array(result["grid_map"])
mobile_pos = result["mobile_pos"]

grid_size = 0.1
floor_pos = [0.0, 0.0]

# Grid dimensions from actual grid_map
height, width = grid_map.shape  # (53, 60)

# Calculate actual floor bounds
floor_half_x = (width * grid_size) / 2  # 60 * 0.1 / 2 = 3.0
floor_half_y = (height * grid_size) / 2  # 53 * 0.1 / 2 = 2.65

floor_min_x = floor_pos[0] - floor_half_x
floor_max_y = floor_pos[1] + floor_half_y

robot_x, robot_y = mobile_pos[0], mobile_pos[1]
robot_grid_j = int(np.floor((robot_x - floor_min_x) / grid_size))
robot_grid_i = int(np.floor((floor_max_y - robot_y) / grid_size))

# Clamp to grid bounds
robot_grid_i = max(0, min(height - 1, robot_grid_i))
robot_grid_j = max(0, min(width - 1, robot_grid_j))

grid_map[robot_grid_i, robot_grid_j] = 2
plt.imshow(grid_map, cmap='viridis')
plt.axis('off')
plt.show()

# Pick and Place

In [ ]:
payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": """
objects = get_object_positions()
RESULT = {
    "objects": objects
}
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
objects = response.json()["result"]["objects"]
objects

In [ ]:
objects["object_fork_0"]

In [ ]:
target_pos = np.array(objects["object_fork_0"]["pos"][:2])

payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": f"""
grid_map = get_grid_map()
path = plan_mobile_path({target_pos.tolist()})
if path is not None:
    for p in path:
        set_mobile_target_joint(p)
RESULT = path
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()

In [ ]:
ee_target_position_1 = np.array(objects["object_fork_0"]["pos"]) + np.array([0.0, 0.0, 0.1])
ee_target_position_2 = np.array(objects["object_fork_0"]["pos"]) + np.array([0.0, 0.0, 0.0])
ee_target_position_3 = np.array(objects["object_fork_0"]["pos"]) + np.array([0.0, 0.0, 0.2])

payload = {"action": {
    "type": "run_code",
    "payload": {
        "code": f"""
result = set_ee_target_position(({ee_target_position_1.tolist()}))
if result:
    result = set_ee_target_position(({ee_target_position_2.tolist()}))
if result:
    result = set_target_gripper_width({0.0})
if result:
    result = set_ee_target_position(({ee_target_position_3.tolist()}))
if result:
    result = set_arm_target_joint([-0.011, -1.032, 0.051, -2.260, 0.063, 1.523, 0.675])
RESULT = result
"""
    }
}}
response = requests.post(f"{url}/send_action", json=payload)
response.json()